# Configure SWASH simulations using the Inductiva API

Now that we've successfuly run a SWASH simulation, let's use the Inductiva API's templating tools to easily programatically configure simulation parameters, and run several simulations in parallel for different input parameters.
Let's start by configuring your API Key and importing the `inductiva` package. We'll also define a util function that will be useful later on.
Note that you must run the API Key cell before the cell that imports the `inductiva` package.

### Tips for running the notebook

For this notebook, you'll need to have a few Python packages installed, which you can do with the following command:

```pip install inductiva scipy numpy matplotlib```

If you're on Google Colab, run the the following cells to install the required dependencies and download the necessary files.

In [ ]:
!pip install inductiva scipy numpy matplotlib
!wget https://storage.googleapis.com/inductiva-api-demo-files/code-blue-hackathon.zip
!unzip code-blue-hackathon.zip

In [ ]:
%env INDUCTIVA_API_KEY=<YOUR_API_KEY_HERE>

In [ ]:
import inductiva

def seconds_to_swash_simulation_time_str(time_s: float):
    """Util function to convert seconds (float) to a string in the format of the swash simulation time.

    SWASH expects the time in the format of "hhmmss.sss", where:
    - hh is the hour
    - mm is the minute
    - ss is the second
    - sss is the millisecond
    """
    m, s = divmod(time_s, 60)
    h, m = divmod(m, 60)
    _, ms = divmod(s, 1)

    return (f"{int(h):02d}"
            f"{int(m):02d}"
            f"{int(s):02d}."
            f"{int(ms*1000):03d}")

### Templating Manager

The Inductiva API integrates a powerful [templating manager](https://tutorials.inductiva.ai/intro_to_api/templating.html) that can be used to programatically change simulation input files. When writing a configuration file, use the notation `{{ parameter_value }}` in place of a parameter that you would like to easily configure. Then, you can use the template manager from Inductiva to create files with specific values of `parameter_value`.

Let's parametrize the coastal area simulation using the templating manager.
Take a look at the template file [input.sws.jinja](assets/coastal_area_simulation_template/input.sws.jinja), which has the template variables `wave_length_m`, `wave_period_s`, and `simulation_time`. The template file, along with the other simulation files (the bathymetry), are place in the `assets/coastal_area_simulation_template` directory.

We can use the `inductiva.TemplateManager.render_dir` method to duplicate the "template directory" with the template parameters replaced by specific values. This allow you to seamlessly launch several simulations and experiment with different configurations.
The `render_dir` method receives as arguments:
- `source_dir`: the path to the template directory
- `tartet_dir`: the path to the directory that will be created by replacing the template variables in the template files
- the template variables expected by your configuration files. In this case, `wave_length_m`, `wave_period_s`, and `simulation_time`.

Note that SWASH expects the simulation time in a "weird" format, so we added a simple util function (`seconds_to_swash_simulation_time_str`) to convert a number of seconds to the expected string.

Let's create input files for simulating 200s of a wave of length 10m (SWASH uses peak-to-peak amplitude) and a period of 10s.

In [16]:
template_dir = "assets/coastal_area_simulation_template"
input_dir = "my_parametrized_inputs"

inductiva.TemplateManager.render_dir(
    source_dir=template_dir,
    target_dir=input_dir,
    wave_length_m=10,
    wave_period_s=10,
    simulation_time=seconds_to_swash_simulation_time_str(200),
)

Now tha we have an input directory with the complete configuration files, we can run a simulation as usual.

In [ ]:
machine_group = inductiva.resources.MachineGroup(machine_type="e2-standard-4", num_machines=1)
machine_group.start()

In [ ]:
swash = inductiva.simulators.SWASH()

task = swash.run(
    input_dir=input_dir,
    sim_config_filename="input.sws",
    on=machine_group,
)

task.wait()

output_path = task.download_outputs()

In [ ]:
machine_group.terminate()

Let's look at the water level in the same point as in the previous notebook.

You can see that the water level and the time reflect the changes made to the input parameters.

In [ ]:
import scipy
import scipy.io
import matplotlib.pyplot as plt

water_level_dict = scipy.io.loadmat(f"{output_path}/water_level.mat")

timesteps = []
water_level_data = []

for key, data in water_level_dict.items():

    if not key.startswith("Watlev"):
        continue

    time_ms = "".join(key.split("_")[1:])

    # Convert time to seconds
    time_s = float(time_ms[:2]) * 3600. + \
        float(time_ms[2:4]) * 60. + \
        float(time_ms[4:6]) +  \
        float(time_ms[6:]) / 1000.

    timesteps.append(time_s)
    water_level_data.append(data.transpose())

def plot_water_level_over_time(water_level_data, timesteps, x, y):
    water_level_xy = [mat[x, y] for mat in water_level_data]

    plt.figure(figsize=(10, 6))
    plt.plot(timesteps, water_level_xy, label=f'Water Level at {(x, y)}')
    plt.xlabel('Time (s)')
    plt.ylabel('Water Level (m)')
    plt.title(f'Water Level Over Time at {(x, y)}')
    plt.legend()
    plt.grid(True)
    plt.show()

plot_water_level_over_time(water_level_data, timesteps, 100, 10)